In [ ]:
# Requires Python 3.10 or higher

In [4]:
#%pip install -r requirements.txt

In [3]:
#%pip install azure-ai-ml

In [1]:
import os
import sys
import time
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

import torch
from awq import AutoAWQForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

from onnxruntime_genai.models.builder import create_model
import onnxruntime_genai as og

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-08 18:34:39.719872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 18:34:40.601372: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 18:34:40.875744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 18:34:42.884907: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow 

In [3]:
experiment_name = "finetuning_experiment"
output_name = "model_dir"
output_directory = "./models"
finetuned_model_path = f"{output_directory}/named-outputs/model_dir/merged"
finetuned_quantized_awq_model_path = f"{output_directory}/named-outputs/model_dir/merged-awq"
finetuned_quantized_onnx_model_path = f"{output_directory}/named-outputs/model_dir/merged-onnx"
finetuned_quantized_awq_to_onnx_model_path = f"{output_directory}/named-outputs/model_dir/merged-awq-onnx"

In [4]:
def generate_output_onnx(model, tokenizer, prompt):
    tokenizer_stream = tokenizer.create_stream()
    input_tokens = tokenizer.encode(prompt)
    
    search_options = {
        "max_length": 2048, 
        "temperature": 0.0, 
        "do_sample": False
    }

    params = og.GeneratorParams(model)
    params.set_search_options(**search_options)
    params.input_ids = input_tokens
    generator = og.Generator(model, params)
    
    started_timestamp = time.time()

    first = True
    new_tokens = []
    new_tokens_decoded = []

    while not generator.is_done():
        generator.compute_logits()
        generator.generate_next_token()
        if first:
            first_token_timestamp = time.time()
            first = False
        new_token = generator.get_next_tokens()[0]
        new_token_decoded = tokenizer_stream.decode(new_token)
        print(new_token_decoded, end='', flush=True)
        new_tokens.append(new_token)
        new_tokens_decoded.append(new_token_decoded)
    
    prompt_length = len(input_tokens)
    new_tones_length = len(new_tokens)
    first_token_time = first_token_timestamp - started_timestamp
    run_time = time.time() - first_token_timestamp
    prompt_tokens_per_second = len(input_tokens)/first_token_time
    new_tokens_per_second = len(new_tokens)/run_time
    
    generated_output = "".join(new_tokens_decoded)
    
    del generator
        
    return generated_output, first_token_time, run_time, prompt_tokens_per_second, new_tokens_per_second

def get_formatted_context(chunks):
    BEGIN = "<DOCUMENT>"
    END = "</DOCUMENT>"
    NEW_LINE = "\n"

    context = [f"{BEGIN}{chunk}{END}{NEW_LINE}" for chunk in chunks]
    return "".join(context)


def get_chat_template_input(meta_prompt, context, query):
    messages = [
        {"content": meta_prompt, "role": "system"},
        {"content": f"{context}{query}.", "role": "user"}
    ]
    return messages

# Substitute "merged" with new transformer model trained with new version

In [7]:
if not os.path.exists(output_directory):
    if not os.listdir(output_directory):   
        try:
            credential = DefaultAzureCredential()
            credential.get_token("https://management.azure.com/.default")
        except Exception as ex:
            # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
            # This will open a browser page for
            credential = InteractiveBrowserCredential()
            
        ml_client = MLClient.from_config(
            credential=credential
        )
        
        os.makedirs(output_directory, exist_ok=True)
        
        jobs = ml_client.jobs.list()

        filtered_jobs = [job for job in jobs if job.experiment_name == experiment_name]
        print(filtered_jobs[-1])
        
        job_name = filtered_jobs[-1].name
        ml_client.jobs.download(name=job_name, output_name=output_name, download_path=output_directory)

In [3]:
def quantize_model(model_path, quant_path, output_path, execution_provider, use_qdq):
    quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

    # Load model
    model = AutoAWQForCausalLM.from_pretrained(
        model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

    # Quantize model
    model.quantize(tokenizer, quant_config=quant_config)

    # Save quantized model
    model.save_quantized(quant_path)
    tokenizer.save_pretrained(quant_path)

    print(f'Model is quantized and saved at "{quant_path}"')

In [8]:
model_path = f"{output_directory}/named-outputs/model_dir/merged"
quant_path = f"{output_directory}/named-outputs/model_dir/merged-awq"
output_path = f"{output_directory}/named-outputs/model_dir/merged-awq-onnx"

execution_provider = "cpu"
use_qdq = True

model_name = None
input_folder = quant_path
output_folder = output_path
precision = "int4"
cache_dir = os.path.join(".", "cache_dir")

In [4]:
quantize_model(model_path, quant_path, output_path, execution_provider, use_qdq)

2024-10-08 18:39:30,195 transformers_modules.microsoft.Phi-3-mini-128k-instruct.a90b62ae09941edff87a90ced39ba5807e6b2ade.modeling_phi3 [WARNING] - `flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
2024-10-08 18:39:30,195 transformers_modules.microsoft.Phi-3-mini-128k-instruct.a90b62ae09941edff87a90ced39ba5807e6b2ade.modeling_phi3 [WARNING] - Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 4/4 [00:49<00:00, 12.50s/it]
Repo card metadata block was not found. Setting CardData to empty.
2024-10-08 18:40:20,847 huggingface_hub.repocard [WARNING] - Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [08:55<00:00, 16.73s/it]
Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


Model is quantized and saved at "./models/named-outputs/model_dir/merged-awq"


In [5]:
extra_options = {
    "use_qdq": use_qdq,
}

create_model(model_name, input_folder, output_folder, precision, execution_provider, cache_dir, **extra_options)

GroupQueryAttention (GQA) is used in this model.


/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:957: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Unpacking and repacking layer 0
Unpacking and repacking layer 1
Unpacking and repacking layer 2
Unpacking and repacking layer 3
Unpacking and repacking layer 4
Unpacking and repacking layer 5
Unpacking and repacking layer 6
Unpacking and repacking layer 7
Unpacking and repacking layer 8
Unpacking and repacking layer 9
Unpacking and repacking layer 10
Unpacking and repacking layer 11
Unpacking and repacking layer 12
Unpacking and repacking layer 13
Unpacking and repacking layer 14
Unpacking and repacking layer 15
Unpacking and repacking layer 16
Unpacking and repacking layer 17
Unpacking and repacking layer 18
Unpacking and repacking layer 19
Unpacking and repacking layer 20
Unpacking and repacking layer 21
Unpacking and repacking layer 22
Unpacking and repacking layer 23
Unpacking and repacking layer 24
Unpacking and repacking layer 25
Unpacking and repacking layer 26
Unpacking and repacking layer 27
Unpacking and repacking layer 28
Unpacking and repacking layer 29
Unpacking and repack

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:924: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:785: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Saving GenAI config in ./models/named-outputs/model_dir/merged-awq-onnx
Saving processing files in ./models/named-outputs/model_dir/merged-awq-onnx for GenAI


In [1]:
import onnxruntime_genai as og

In [5]:
def create_prompt(chat_template, meta_prompt, query, context):
    prompt = f'{chat_template.format(meta_prompt=meta_prompt, context=context, query=query)}'
    return prompt

meta_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers."
chunks = [
    "Various. Verify the error codes in the balance\u2019s manual.MAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n29BASIC DEFINITIONS\nASTM. American Society of Testing and Materials.",
    "Preventive maintenance\nFrequency: Quarterly1. Verify the stability of the lamp. Use the calibration plate, \nconducting readings with intervals of 30 minutes with the same plate. Compare readings. There must be no diff erences.",
    "2. A measuring device known as \u201cload cell\u201d produces an \nexit signal corresponding to the load\u2019s force in the form of changes in the voltage or frequency. 3. A digital analogous electronic circuit shows the fi  nal \nresult of the weight digitally. Laboratory balances operate according to the principle \nof compensation of the electromagnetic force applicable to displacements or torques. The combination of their mechanical components and automatic reading systems provides weight measurements at defi  ned levels of accuracy \ndepending on the model. Principle. The mobile parts (weighing plate, support \ncolumn [a], bobbin, position and load indicator [G] -the object in the process of being weighed-) are maintained in equilibrium by a compensation force [F] equal to the weight. The compensation force is generated by an electrical current through a bobbin in the air gap of a cylindrical electromagnet. The force F is calculated with the equation [F = I x L x B] where: I = electrical intensity, L = total length of the wire of the coil and B = magnetic fl  ow intensity in the \nelectromagnet\u2019s air gap.With any change in the load (weight\/mass), the mobile \nmechanical system responds by moving vertically a fraction of distance. Detected by a photosensor [e], an electrical signal is sent to the servo-amplifi  er [f]. This changes the \nfl ow of electrical current passing through the bobbin of the \nmagnet [c] in such a manner that the mobile system returns to the balanced position upon adjusting of the magnetic fl ow in the electromagnet. Consequently, the weight of \nthe mass [G] can be measured indirectly at the start of the electrical current fl  ow, which passes through the circuit \nmeasuring the voltage [V] by means of a precision resistor [R], [V = I x R]. To date, many systems developed use the electronic system for carrying out very exact measurements of mass and weight. The following diagram explains how electronic balances function. Transfer\nMechanism\nLoad Cell\nScreen and\nSignal ProcessorPFigure 12. Components of electronic balances  \nG\nb\na\ne\nfc dR V=I*R\nIFigure 13. Compensation force principle  \nMAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n25The signal processing system\nThe signal processing system is composed of the circuit which \ntransf orms the electrical signal emitted by the transducer \ninto numerical data which can be read on a screen. The signal process comprises the following functions:1. Tare setting. This setting is used to adjust the reading \nvalue at zero with any load within the balance\u2019s capacity range. It is controlled by a button generally located on the front part of the balance. It is commonly used for taring the weighing container. 2. Repeatability setting control. During a reading, weighed \nvalues are averaged within a predefi  ned period of time. This function is very useful when weighing operations need to be carried out in unstable conditions, e.g. in the presence of air currents or vibrations. This control defi nes the time period allowed for a result to lie within \npreset limits for it to be considered stable. In addition, it can be adjusted to suit a particular application.",
    "Any spill must be cleaned immediately to avoid corrosion \nor contamination. Use 70% ethanol to disinfect the pan of the balance. Very important:  Never lubricate a balance unless the \nmanufacturer has expressly indicated it. Any substance interfering with the mechanism of the balance retards its response or defi  nitely alters the measurement process. Note:  In general, the manufacturer or the specialized \ninstallation representative carries out the maintenance of the balances, according to procedures which vary depending on the type and model. 1 Guidelines for calibration in laboratories, Drinking Water Inspectorate by \nLGC (Teddington) Ltd., December 2000. CapacityResolution\n100 g 10 g 1 g 100 mg 10 mg 1 mg 0.1 mg \u00980.01 mg \nUp to 200 g \u2013 \u2013 \u2013 M1 M1 F2 F1 F2\n200 g to 1 kg \u2013 \u2013 M1 M1 F2 F1\/E2 E2 E2\n1 to 30 kg M2 M2 M1 F2 E2 E2 E2 \u2013\n30 to 100 kg M2 M1 F2 F1 E2 \u2013 \u2013 \u2013\nMore than \n100 kgM2 M1\/F2 F1 E2 \u2013 \u2013 \u2013 \u2013Table of standard weights\u2019 use according to the balance\u2019s capacity  CHAPTER 4  BALANCES\n28FUNCTIONAL ERROR PROBABLE CAUSE\nReadings not reproducible (hysteresis). The measurement cell is dirty.",
    "Check the lubrication state of elements such as for \nO-rings as the manufacturer recommends. Always use lubricants according to the manufacturer\u2019s instructions (frequency and type of lubricants). In recently manufactured centrifuges, there are sealed ball bearings which do not require lubrication. 5.",
    "Remove the cover of the boiling tank.3. Visually verify if the interior walls or the immersion \nresistors show solid deposits or sediments. The quantity of deposits present depends on the quality of water fed to the distiller. If there is an accumulation of sediments, it must be cleaned to avoid damaging the resistors\n1. 4."
]
context = get_formatted_context(chunks)
query = "What does ASTM stand for?"
chat_template = "<|system|>\n{meta_prompt}<|end|>\n<|user|>\n{context}{query}<|end|>\n<|assistant|>"

prompt = create_prompt(chat_template, meta_prompt, query, context)
prompt

'<|system|>\nThe following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers.<|end|>\n<|user|>\n<DOCUMENT>Various. Verify the error codes in the balance’s manual.MAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n29BASIC DEFINITIONS\nASTM. American Society of Testing and Materials.</DOCUMENT>\n<DOCUMENT>Preventive maintenance\nFrequency: Quarterly1. Verify the stability of the lamp. Use the calibration plate, \nconducting readings with intervals of 30 minutes with the same plate. Compare readings. There must be no diff erences.</DOCUMENT>\n<DOCUMENT>2. A measuring device known as “load cell” produces an \nexit signal corresponding to the load’s force in the form of changes in the voltage or frequency. 3. A digital analogous electronic circuit shows the fi  nal \nresult of the weight digitally. Laboratory balances operate according to the principle \nof compensation of the electromagnetic force applicable to displacements

In [6]:
def load_model(output_path):
    model = og.Model(output_path)
    tokenizer = og.Tokenizer(model)    
    return model, tokenizer


def run_model(model, tokenizer, prompt):
    tokenizer_stream = tokenizer.create_stream()

    # Override any default search options in `genai_config.json`
    search_options = {
        'min_length': 1,
        'max_length': 2048,
    }

    input_tokens = tokenizer.encode(prompt)

    params = og.GeneratorParams(model)
    params.set_search_options(**search_options)
    params.input_ids = input_tokens

    generator = og.Generator(model, params)

    print()
    print("Output: ", end='', flush=True)

    try:
        while not generator.is_done():
            generator.compute_logits()
            generator.generate_next_token()

            new_token = generator.get_next_tokens()[0]
            print(tokenizer_stream.decode(new_token), end='', flush=True)
    except KeyboardInterrupt:
        print("  --control+c pressed, aborting generation--")

    # Delete the generator to free the captured graph for the next generator, if graph capture is enabled
    del generator

In [9]:
quantized_model, tokenizer = load_model(output_path)

In [10]:
run_model(quantized_model, tokenizer, prompt)


Output:  To answer the question "What does ASTM stand for?", I will follow these steps:

1. Identify the acronym "ASTM" in the context provided.
2. Look for any definitions or explanations related to "ASTM" in the context.
3. Extract the relevant information that defines "ASTM."

From the context, I see the following relevant information:

##begin_quote##
"ASTM."
##end_quote##

Since the context does not provide a direct definition, I will refer to the general knowledge that ASTM stands for "American Society for Testing and Materials."

Final answer: <ANSWER>: ASTM stands for American Society for Testing and Materials.

In [23]:
pytorch_model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_path,
    device_map="auto"
)
pytorch_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)

2024-10-08 19:07:39,254 accelerate.utils.modeling [INFO] - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 4/4 [01:52<00:00, 28.13s/it]


In [7]:
meta_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers."
chunks = [
    "Various. Verify the error codes in the balance\u2019s manual.MAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n29BASIC DEFINITIONS\nASTM. American Society of Testing and Materials.",
    "Preventive maintenance\nFrequency: Quarterly1. Verify the stability of the lamp. Use the calibration plate, \nconducting readings with intervals of 30 minutes with the same plate. Compare readings. There must be no diff erences.",
    "2. A measuring device known as \u201cload cell\u201d produces an \nexit signal corresponding to the load\u2019s force in the form of changes in the voltage or frequency. 3. A digital analogous electronic circuit shows the fi  nal \nresult of the weight digitally. Laboratory balances operate according to the principle \nof compensation of the electromagnetic force applicable to displacements or torques. The combination of their mechanical components and automatic reading systems provides weight measurements at defi  ned levels of accuracy \ndepending on the model. Principle. The mobile parts (weighing plate, support \ncolumn [a], bobbin, position and load indicator [G] -the object in the process of being weighed-) are maintained in equilibrium by a compensation force [F] equal to the weight. The compensation force is generated by an electrical current through a bobbin in the air gap of a cylindrical electromagnet. The force F is calculated with the equation [F = I x L x B] where: I = electrical intensity, L = total length of the wire of the coil and B = magnetic fl  ow intensity in the \nelectromagnet\u2019s air gap.With any change in the load (weight\/mass), the mobile \nmechanical system responds by moving vertically a fraction of distance. Detected by a photosensor [e], an electrical signal is sent to the servo-amplifi  er [f]. This changes the \nfl ow of electrical current passing through the bobbin of the \nmagnet [c] in such a manner that the mobile system returns to the balanced position upon adjusting of the magnetic fl ow in the electromagnet. Consequently, the weight of \nthe mass [G] can be measured indirectly at the start of the electrical current fl  ow, which passes through the circuit \nmeasuring the voltage [V] by means of a precision resistor [R], [V = I x R]. To date, many systems developed use the electronic system for carrying out very exact measurements of mass and weight. The following diagram explains how electronic balances function. Transfer\nMechanism\nLoad Cell\nScreen and\nSignal ProcessorPFigure 12. Components of electronic balances  \nG\nb\na\ne\nfc dR V=I*R\nIFigure 13. Compensation force principle  \nMAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n25The signal processing system\nThe signal processing system is composed of the circuit which \ntransf orms the electrical signal emitted by the transducer \ninto numerical data which can be read on a screen. The signal process comprises the following functions:1. Tare setting. This setting is used to adjust the reading \nvalue at zero with any load within the balance\u2019s capacity range. It is controlled by a button generally located on the front part of the balance. It is commonly used for taring the weighing container. 2. Repeatability setting control. During a reading, weighed \nvalues are averaged within a predefi  ned period of time. This function is very useful when weighing operations need to be carried out in unstable conditions, e.g. in the presence of air currents or vibrations. This control defi nes the time period allowed for a result to lie within \npreset limits for it to be considered stable. In addition, it can be adjusted to suit a particular application.",
    "Any spill must be cleaned immediately to avoid corrosion \nor contamination. Use 70% ethanol to disinfect the pan of the balance. Very important:  Never lubricate a balance unless the \nmanufacturer has expressly indicated it. Any substance interfering with the mechanism of the balance retards its response or defi  nitely alters the measurement process. Note:  In general, the manufacturer or the specialized \ninstallation representative carries out the maintenance of the balances, according to procedures which vary depending on the type and model. 1 Guidelines for calibration in laboratories, Drinking Water Inspectorate by \nLGC (Teddington) Ltd., December 2000. CapacityResolution\n100 g 10 g 1 g 100 mg 10 mg 1 mg 0.1 mg \u00980.01 mg \nUp to 200 g \u2013 \u2013 \u2013 M1 M1 F2 F1 F2\n200 g to 1 kg \u2013 \u2013 M1 M1 F2 F1\/E2 E2 E2\n1 to 30 kg M2 M2 M1 F2 E2 E2 E2 \u2013\n30 to 100 kg M2 M1 F2 F1 E2 \u2013 \u2013 \u2013\nMore than \n100 kgM2 M1\/F2 F1 E2 \u2013 \u2013 \u2013 \u2013Table of standard weights\u2019 use according to the balance\u2019s capacity  CHAPTER 4  BALANCES\n28FUNCTIONAL ERROR PROBABLE CAUSE\nReadings not reproducible (hysteresis). The measurement cell is dirty.",
    "Check the lubrication state of elements such as for \nO-rings as the manufacturer recommends. Always use lubricants according to the manufacturer\u2019s instructions (frequency and type of lubricants). In recently manufactured centrifuges, there are sealed ball bearings which do not require lubrication. 5.",
    "Remove the cover of the boiling tank.3. Visually verify if the interior walls or the immersion \nresistors show solid deposits or sediments. The quantity of deposits present depends on the quality of water fed to the distiller. If there is an accumulation of sediments, it must be cleaned to avoid damaging the resistors\n1. 4."
]
context = get_formatted_context(chunks)
query = "What does ASTM stand for?"
messages = get_chat_template_input(meta_prompt, context, query)

inputs = pytorch_tokenizer.apply_chat_template(
  messages,
  tokenize=True,
  add_generation_prompt=True,
  return_tensors="pt",
  return_dict=True,
).to("cuda")

outputs = pytorch_model.generate(**inputs, do_sample=True, max_new_tokens=512)
print(pytorch_tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0])

NameError: name 'pytorch_tokenizer' is not defined

In [13]:
# NOW BENCHMARK:
# https://github.com/microsoft/onnxruntime/blob/main/onnxruntime/python/tools/transformers/models/llama/benchmark_e2e.py